NetworkX-Temporal
---

- Links:
[Documentation](https://networkx-temporal.readthedocs.io/en/stable/examples/basics.html) |
[PyPI project](https://pypi.org/p/networkx-temporal/) |
[GitHub repository](https://github.com/nelsonaloysio/networkx-temporal)

- Examples:
[Basic operations](networkx-temporal-01-basics.ipynb) |
[Convert and transform](networkx-temporal-02-convert.ipynb) |
[Algorithms and metrics](networkx-temporal-03-metrics.ipynb) |
[Community detection](networkx-temporal-04-community.ipynb)

In [ ]:
!pip install -q 'networkx-temporal[ipynb]'   # Installs additional libraries used in this notebook.

___

# Convert and transform

This package provides a set of functions to manipulate graph classes,
formats, and representations. In this context,
``convert`` refers to different graph-based
libraries, e.g., [igraph](https://igraph.org/python), and
``transform`` refers to the underlying data structure
used to store object relations, e.g., [event-based temporal
graphs](#event-based-temporal-graph).

## Graph types and classes

The ``transform`` module provides functions to convert among different
``TemporalGraph`` types, depending on whether the underlying data
structure allows parallel edges (multigraphs) or not.
Static or temporal multigraphs may be converted to graphs without parallel edges and vice-versa.

In [ ]:
>>> %load_ext autoreload
>>> %autoreload 2
>>> import networkx_temporal as tx
>>>
>>> TG = tx.temporal_graph(directed=True)  # tx.TemporalMultiDiGraph
>>>
>>> TG.add_edge("a", "b", time=0)
>>> TG.add_edge("c", "b", time=1)
>>> TG.add_edge("d", "c", time=2)
>>> TG.add_edge("d", "e", time=2)
>>> TG.add_edge("a", "c", time=2)
>>> TG.add_edge("f", "e", time=3)
>>> TG.add_edge("f", "a", time=3)
>>> TG.add_edge("f", "b", time=3)
>>>
>>> print(TG)

In [ ]:
>>> TG.add_edge("c", "b", time=0)   # <-- parallel edge
>>> print(TG)

The `from_multigraph` function combines parallel edges
and sum their ``weight`` (default: $1$) values. Notice that $(c, b)$ ``time``
is now set to $0$, as later attribute values take precedence over earlier ones:


In [ ]:
>>> TG = tx.from_multigraph(TG)
>>> TG.edge("c", "b")

Converting the resulting
``TemporalDiGraph``
back to a
``TemporalMultiDiGraph``
does not restore data:

In [ ]:
>>> TG = tx.to_multigraph(TG)
>>> print(TG)

In [ ]:
>>> TG.add_edge("c", "b", time=1)   # <-- parallel edge
>>> print(TG)

## Graph representations

Once instantiated, ``TemporalGraph`` objects
may be transformed into different representations, depending on the
analysis or visualization requirements. Due to the nature of temporal
graphs, some representations may not preserve all the data, such as
dynamic node or edge attributes.

Observe that the total number of returned nodes $V$ and edges $E$ after
transformation might differ from the number of temporal nodes $V_T$ and
edges $E_T$, depending on the data and method used:

| Representation | Order | Size  | Dynamic node attributes | Dynamic edge attributes |
| --- | :---: | :---: | :---: | :---: |
| [Static](#static-graph) | $V = V_T$ | $E = E_T$ | ❌ | ✅ |
| [Snapshots](#snapshot-based-temporal-graph)* | $V \ge V_T$ | $E = E_T$ | ✅ | ✅ |
| [Events](#event-based-temporal-graph) | $V = V_T$ | $E = E_T$ | ❌ | ❌ |
| [Unrolled](#unrolled-temporal-graph) | $V \ge V_T$ | $E \ge E_T$ | ✅ | ✅ |

(\*) Default underlying data structure for temporal graphs with multiple
snapshots on ``slice``.

### Static graph

A static graph `G` is a single graph object containing all the nodes and
edges found in the temporal graph. It is the simplest representation of
a network and is the most common type of graph.

> Dynamic node attributes are not preserved when transforming a temporal
> to a static graph.

> See also:
> The [Basic operations → Import static
> graphs](basics.html#import-static-graphs) page for more static graph
> conversion examples.

#### `TG` → `G`

Transforming a ``TemporalGraph`` into a static
graph with the ``to_static``
method:


In [ ]:
>>> G = TG.to_static()
>>> print(G)

In [ ]:
>>> tx.draw(G, layout="kamada_kawai", suptitle="Static Graph")

#### `G` → `TG`

Transforming a static graph into a
``TemporalGraph`` with the
``from_static`` function:

In [ ]:
>>> TG = tx.from_static(G)
>>> TG = TG.slice(attr="time")
>>> print(TG)

### Snapshot-based temporal graph

A snapshot-based temporal graph `STG` is a sequence of graphs where each
element represents a snapshot of the original temporal graph. It is the
most common representation of temporal graphs.

> Note:
> Like the ``slice`` method,
> ``to_snapshots`` internally
> returns views of the original graph data, so no data is copied unless
> specified otherwise, i.e., by passing `as_view=False` to the function.

#### `TG` → `STG`

Transforming a ``TemporalGraph`` into a
snapshot-based temporal graph with
``to_snapshots``:


In [ ]:
>>> STG = TG.to_snapshots()
>>> STG

#### `STG` → `TG`

Transforming a snapshot-based temporal graph into a
``TemporalGraph`` with
``from_snapshots``:

In [ ]:
>>> TG = tx.from_snapshots(STG).copy()
>>> print(TG)

### Event-based temporal graph

An event-based temporal graph `ETG` is a sequence of 3- or 4-tuple
edge-based events.

- **3-tuples** ($u, v, t$), where elements are the source node, target
  node, and time attribute;
- **4-tuples** ($u, v, t, \delta$), where an additional element $\delta$
  is either an `int` for edge addition (`1`) or deletion (`-1`) events,
  or a `float` for the duration of the interaction (zero for a single
  snapshot).

Depending on the temporal graph data, one of these may allow a more
compact representation than the others. The default is to return a
3-tuple sequence (also known as a *stream graph*).

> As events are edge-based, node isolates without self-loops are not
> preserved.

#### `TG` → `ETG`

Transforming a ``TemporalGraph`` into an
event-based temporal graph with
``to_events``:


In [ ]:
>>> ETG = TG.to_events()  # delta=None
>>> ETG

In [ ]:
>>> ETG = TG.to_events(delta=int)
>>> ETG

In [ ]:
>>> ETG = TG.to_events(delta=float)
>>> ETG

#### `ETG` → `TG`

Node and edge attributes are not preserved when transforming a graph to a sequence of events,
but topological information is retained, allowing to reconstruct its snapshots with
``from_events``:

In [ ]:
>>> TG = tx.from_events(ETG, directed=True, multigraph=True)
>>> print(TG)

### Unrolled temporal graph

An unrolled temporal graph `UTG` is a single graph object that contains
the original temporal data, plus additional time-adjacent node copies
(from each snapshot) and edge couplings connecting them. It is mainly
useful for certain analysis and visualization tasks, e.g., based on
temporal flows.

For an example with temporal node centrality metrics, see [Hyoungshick &
Anderson, 2012](https://doi.org/10.1103/PhysRevE.85.026107)[1].

#### `TG` → `UTG`

Transforming a ``TemporalGraph`` into an
unrolled temporal graph with
``to_unrolled``:

In [ ]:
>>> UTG = TG.to_unrolled(edge_couplings=True)
>>> print(UTG)

Let's draw the resulting graph to visualize the node copies (in black)
and edge couplings (dotted):

In [ ]:
>>> def draw_unrolled(UTG, **kwargs):
>>>     return tx.draw(
>>>         UTG,
>>>         layout=tx.unrolled_layout,
>>>         labels={n: f"{n.split('_')[0]}$_{n.split('_')[1]}$" for n in UTG.nodes()},
>>>         font_size=10,
>>>         arrowsize=15,
>>>         **kwargs,
>>>     )
>>>
>>> node_color = [
>>>     "tab:red" if int(n.split("_")[1]) == TG.index_node(n.split("_")[0])[0] else "#333"
>>>     for n in UTG.nodes()]
>>>
>>> draw_unrolled(UTG,
>>>               node_color=node_color,
>>>               connectionstyle="arc3,rad=0.25",
>>>               title="Unrolled Temporal Graph")

> By default, edges connect nodes in the same snapshot, e.g., from $u_t$
> to $v_t$. To create edges that connect nodes across time, e.g., from
> $u_t$ to $v_{t+\delta}$, pass the `delta` parameter to the function with
> the desired edge-level attribute, e.g., `delta='duration'`, or time
> difference, e.g., `delta=1`.

Passing `delta=1` to the function creates edges connecting nodes in
adjacent snapshots. In the following plot, nodes not present in the
previous example are colored in black:

In [ ]:
>>> UTG_delta = TG.to_unrolled(delta=1)
>>>
>>> node_color = [
>>>     "tab:red" if UTG.has_node(n) else "#333"
>>>     for n in UTG_delta.nodes()]
>>>
>>> draw_unrolled(UTG_delta,
>>>               node_color=node_color,
>>>               title="Unrolled Temporal Graph ($\\delta=1$)")

> New nodes and edges are created depending on the `delta` value passed to
> the function, leading to graphs of different order and size. For
> instance, passing `delta=1` in the example above created additional
> edges among node $f_3$ and the temporal node copies $a_4$, $b_4$, and
> $e_4$.

Lastly, the additional parameters `edge_couplings` and `node_copies`
allow further control over the creation of temporal node copies and edge
couplings. A comparison with newly added nodes in red:

In [ ]:
>>> import matplotlib.pyplot as plt
>>>
>>> fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(8, 2))
>>>
>>> UTG_fill = TG.to_unrolled(delta=1, node_copies="fill")
>>> UTG_persist = TG.to_unrolled(delta=1, node_copies="persist")
>>> UTG_all = TG.to_unrolled(delta=1, node_copies="all")
>>>
>>> draw_unrolled(
>>>     UTG_fill,
>>>     node_size=200, fig=fig, ax=0, title="node_copies='fill'")
>>>
>>> draw_unrolled(
>>>     UTG_persist,
>>>     node_color=["#333" if UTG_fill.has_node(n) else "tab:red" for n in UTG_persist.nodes()],
>>>     node_size=200, fig=fig, ax=1, title="node_copies='persist'")
>>>
>>> draw_unrolled(
>>>     UTG_all,
>>>     node_color=["#333" if UTG_persist.has_node(n) else "tab:red" for n in UTG_all.nodes()],
>>>     node_size=200, fig=fig, ax=2, title="node_copies='all'")


#### `UTG` → `TG`

As with events, node and edge attributes are not preserved when unrolling and rerolling graphs,
but their structural information is retained.
Obtanining a
``TemporalGraph`` with
``from_unrolled``:

In [ ]:
>>> TG = tx.from_unrolled(UTG)
>>> print(TG)

## External library formats

Support for the following external libraries is currently implemented
in the package:

| Format | Parameter (Package) | Calls (Function) |
|---|:---:|:---:|
| [Deep Graph Library](https://www.dgl.ai) | `'dgl'` | [`to_dgl`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_dgl)
| [DyNetX](https://dynetx.readthedocs.io) | `'dynetx'` | [`to_dynetx`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_dynetx)
| [graph-tool](https://graph-tool.skewed.de) | `'graph_tool'` | [`to_graph_tool`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_graph_tool)
| [igraph](https://igraph.org/python) | `'igraph'` | [`to_igraph`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_igraph)
| [NetworKit](https://networkit.github.io) | `'networkit'` | [`to_networkit`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_networkit)
| [NumPy](https://numpy.org) | `'numpy'` | [`to_numpy`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_numpy)
| [PyTorch Geometric](https://pytorch-geometric.readthedocs.io) | `'torch_geometric'` | [`to_torch_geometric`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_torch_geometric)
| [SciPy](https://scipy.org) | `'scipy'` | [`to_scipy`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_scipy)
| [SNAP](https://snap.stanford.edu) | `'snap'` | [`to_snap`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_snap)
| [StellarGraph](https://stellargraph.readthedocs.io) | `'stellargraph'` | [`to_stellargraph`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_stellargraph)
| [Teneto](https://teneto.readthedocs.io) | `'teneto'` | [`to_teneto`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_teneto)

Graphs may be converted to a different library format with the
high-level ``convert`` function:

In [ ]:
>>> tx.convert(TG, "igraph")

By default, the amount of objects returned match the number of slices
(snapshots). To return a single object containing all the nodes and
edges found in the temporal graph:

In [ ]:
>>> tx.convert(TG.to_static(), "igraph")

## File readers and writers

The ``read_graph`` and
``write_graph`` functions offer a high-level
interface to load and store graph data, supporting any format
implemented in the current installed version of NetworkX.

In [ ]:
>>> tx.write_graph(TG, "temporal-graph.graphml.zip")
>>> TG = tx.read_graph("temporal-graph.graphml.zip")
>>> print(TG)

File formats supported by the installed version of NetworkX may be used to read and write temporal graph data, including GML, GEXF, GraphML, Pajek, LEDA, and adjacency list formats.
Input and output may be omitted or fed directly to the functions as well:

In [ ]:
>>> byte_data = tx.write_graph(TG, format="graphml")
>>> TG = tx.read_graph(byte_data)
>>> print(TG)

> See: The [read and write
> documentation](https://networkx.org/documentation/stable/reference/readwrite/index.html)
> from NetworkX for a list of supported formats and details.

------------------------------------------------------------------------

[1] Hyoungshick, K., Anderson, R. (2012). ''Temporal node centrality in
complex networks''. Physical Review E, 85(2).